In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('Training.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.columns = df.columns.str.replace('_', ' ', regex=False)

In [5]:
df.head()

,itching,skin rash,nodal skin eruptions,continuous sneezing,shivering,chills,joint pain,stomach pain,acidity,ulcers on tongue,muscle wasting,vomiting,burning micturition,spotting urination,fatigue,weight gain,anxiety,cold hands and feets,mood swings,weight loss,restlessness,lethargy,patches in throat,irregular sugar level,cough,high fever,sunken eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish skin,dark urine,nausea,loss of appetite,pain behind the eyes,back pain,constipation,abdominal pain,diarrhoea,mild fever,yellow urine,yellowing of eyes,acute liver failure,fluid overload,swelling of stomach,swelled lymph nodes,malaise,blurred and distorted vision,phlegm,throat irritation,redness of eyes,sinus pressure,runny nose,congestion,chest pain,weakness in limbs,fast heart rate,pain during bowel movements,pain in anal region,bloody stool,irritation in anus,neck pain,dizziness,cramps,bruising,obesity,swollen legs,swollen blood vessels,puffy face and eyes,enlarged thyroid,brittle nails,swollen extremeties,excessive hunger,extra marital contacts,drying and tingling lips,slurred speech,knee pain,hip joint pain,muscle weakness,stiff neck,swelling joints,movement stiffness,spinning movements,loss of balance,unsteadiness,weakness of one body side,loss of smell,bladder discomfort,foul smell of urine,continuous feel of urine,passage of gases,internal itching,toxic look (typhos),depression,irritability,muscle pain,altered sensorium,red spots over body,belly pain,abnormal menstruation,dischromic patches,watering from eyes,increased appetite,polyuria,family history,mucoid sputum,rusty sputum,lack of concentration,visual disturbances,receiving blood transfusion,receiving unsterile injections,coma,stomach bleeding,distention of abdomen,history of alcohol consumption,fluid overload.1,blood in sputum,prominent veins on calf,palpitations,painful walking,pus filled pimples,blackheads,scurring,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 5.0+ MB


In [7]:
df.isnull().sum()

itching                 0
skin rash               0
nodal skin eruptions    0
continuous sneezing     0
shivering               0
                       ..
inflammatory nails      0
blister                 0
red sore around nose    0
yellow crust ooze       0
prognosis               0
Length: 133, dtype: int64

In [8]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [9]:
le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
column_dict = {df.columns[i]: i for i in range(132)}

In [11]:
disease_name = pd.concat([pd.DataFrame(y), df['prognosis']],axis = 1)

In [12]:
disease_name.columns = ['encoded','prognosis']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 20)

In [14]:
svc = SVC(kernel = 'linear',probability=True)


svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print("r2 score",r2_score(y_test, y_pred))

r2 score 1.0


In [16]:
def prediction(user_symptoms):

    input_vector = np.zeros(len(column_dict))
    
    for item in user_symptoms:
        
        input_vector[column_dict[item]] = 1

    result = svc.predict_proba([input_vector])[0]
    
    print('====================Disease=========================')
    result = np.argsort(result)[::-1][:3]
    check = svc.classes_[result]
    disease = disease_name[disease_name['encoded']==check[2]]['prognosis'].values[0]
    print(disease)
    # return helper(disease)
    

In [17]:
prediction(['itching'])

====================Disease=========================
Heart attack


C:\Users\mdzai\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [17]:
precautions = pd.read_csv('precautions_df.csv',usecols = lambda x : not x.startswith('Unnamed'))

In [18]:
precautions.loc[2,'Precaution_3'] = 'Avoid Further Irritants'

In [19]:
precautions.loc[36,'Precaution_4'] = 'Avoid panic and unnecessary movement'

In [20]:
description = pd.read_csv('description.csv',usecols = lambda x : not x.startswith('Unnamed'))

In [21]:
diet = pd.read_csv('diets.csv')

In [22]:
import ast
diet["Diet"] = diet["Diet"].apply(ast.literal_eval)

In [23]:
workouts = pd.read_csv('workout_df.csv',usecols = lambda x : not x.startswith('Unnamed'))

In [24]:
medications = pd.read_csv('medications.csv')

In [25]:
import ast
medications["Medication"] = medications["Medication"].apply(ast.literal_eval)

In [76]:
def helper(predicted_dis):

    pre = precautions[precautions['Disease']==predicted_dis][['Precaution_1','Precaution_2','Precaution_3','Precaution_4']].values[0]

    des = description[description['Disease']==predicted_dis]['Description'].values[0]

    diets = diet[diet['Disease']==predicted_dis]['Diet'].values[0]

    workout = workouts[workouts['disease']==predicted_dis]['workout'].values

    med = medications[medications['Disease']==predicted_dis]['Medication'].values[0]


    print("=======================description====================")
    
    print("==>",des)
    
    print("=======================precautions====================")
    
    count=1
    for i in this:
        print(count,")",i)
        count+=1
        
    print("=======================diets====================")

    count=1
    for i in diets:
        print(count,")",i)
        count+=1
    
    print("=======================workout====================")

    count=1
    for i in workout:
        print(count,")",i)
        count+=1
    
    print("=======================medication====================")

    count=1
    for i in med:
        print(count,")",i)
        count+=1

In [79]:
prediction(['skin rash','joint pain','skin peeling'])

====================Disease=========================
Psoriasis
=======================description====================
==> Psoriasis is a chronic skin condition characterized by red, itchy, and scaly patches.
=======================precautions====================
1 ) apply calamine
2 ) cover area with bandage
3 ) Avoid Further Irritants
4 ) use ice to compress itching
=======================diets====================
1 ) Psoriasis Diet
2 ) Anti-Inflammatory Diet
3 ) Omega-3-rich foods
4 ) Fruits and vegetables
5 ) Whole grains
=======================workout====================
1 ) Consume anti-inflammatory foods
2 ) Include omega-3 fatty acids
3 ) Include vitamin D analogues
4 ) Limit alcohol consumption
5 ) Stay hydrated
6 ) Consult a healthcare professional
7 ) Limit processed foods
8 ) Follow medical recommendations
9 ) Manage stress
10 ) Consider phototherapy under medical guidance
=======================medication====================
1 ) Topical treatments
2 ) Phototherapy
3 ) Syste

C:\Users\mdzai\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [91]:
import pickle
pickle.dump(svc,open('svc.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(precautions,open('precautions.pkl','wb'))
pickle.dump(description,open('description.pkl','wb'))
pickle.dump(diet,open('diet.pkl','wb'))
pickle.dump(workouts,open('workouts.pkl','wb'))
pickle.dump(disease_name,open('disease_name.pkl','wb'))
pickle.dump(medications,open('medications.pkl','wb'))

In [92]:
symptoms = pd.DataFrame(df.columns, columns=['symptoms'])

In [93]:
pickle.dump(symptoms,open('symptoms.pkl','wb'))

In [90]:
symptoms

,symptoms
0,itching
1,skin rash
2,nodal skin eruptions
3,continuous sneezing
4,shivering
...,...
128,inflammatory nails
129,blister
130,red sore around nose
131,yellow crust ooze


In [98]:
df[df['prognosis']=='Typhoid']

,itching,skin rash,nodal skin eruptions,continuous sneezing,shivering,chills,joint pain,stomach pain,acidity,ulcers on tongue,muscle wasting,vomiting,burning micturition,spotting urination,fatigue,weight gain,anxiety,cold hands and feets,mood swings,weight loss,restlessness,lethargy,patches in throat,irregular sugar level,cough,high fever,sunken eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish skin,dark urine,nausea,loss of appetite,pain behind the eyes,back pain,constipation,abdominal pain,diarrhoea,mild fever,yellow urine,yellowing of eyes,acute liver failure,fluid overload,swelling of stomach,swelled lymph nodes,malaise,blurred and distorted vision,phlegm,throat irritation,redness of eyes,sinus pressure,runny nose,congestion,chest pain,weakness in limbs,fast heart rate,pain during bowel movements,pain in anal region,bloody stool,irritation in anus,neck pain,dizziness,cramps,bruising,obesity,swollen legs,swollen blood vessels,puffy face and eyes,enlarged thyroid,brittle nails,swollen extremeties,excessive hunger,extra marital contacts,drying and tingling lips,slurred speech,knee pain,hip joint pain,muscle weakness,stiff neck,swelling joints,movement stiffness,spinning movements,loss of balance,unsteadiness,weakness of one body side,loss of smell,bladder discomfort,foul smell of urine,continuous feel of urine,passage of gases,internal itching,toxic look (typhos),depression,irritability,muscle pain,altered sensorium,red spots over body,belly pain,abnormal menstruation,dischromic patches,watering from eyes,increased appetite,polyuria,family history,mucoid sputum,rusty sputum,lack of concentration,visual disturbances,receiving blood transfusion,receiving unsterile injections,coma,stomach bleeding,distention of abdomen,history of alcohol consumption,fluid overload.1,blood in sputum,prominent veins on calf,palpitations,painful walking,pus filled pimples,blackheads,scurring,skin peeling,silver like dusting,small dents in nails,inflammatory nails,blister,red sore around nose,yellow crust ooze,prognosis
180,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Typhoid
181,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Typhoid
182,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Typhoid
183,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Typhoid
184,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Typhoid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4733,0,0,0,0,0,1,0,0,0,0,0,1,

In [18]:
import pickle
pickle.dump(svc,open('svc_p.pkl','wb'))